In [28]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import data_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from datetime import datetime
import pickle

import lightgbm as lgb
import pandas as pd
import numpy as np

SEED = 42

In [30]:
X = pd.read_pickle("../data/intermediate/X.pkl")
Y = pd.read_pickle("../data/intermediate/Y.pkl")

In [31]:
# need to convert all categorical to numbers for sklearn SVM
# first convert the nans in our categorical back to "?", will use as category
X = X.apply(lambda x: x.cat.add_categories("?").fillna("?") if x.dtype.name == 'category' and not "?" in x.cat.categories.values else x)

In [32]:
X = pd.get_dummies(data=X, columns=X.columns[X.dtypes == 'category'], drop_first=True)

### Feature Selection

In [33]:
field_descriptions_fpath = data_tools.field_descriptions_fpath
fields = data_tools.parse_field_descriptions(field_descriptions_fpath)

In [34]:
# Use this to select features
# currently using all features so ignore this, probably want to do this before we extend the categorical vars above to make it easier
selected_features = None

### Validation

In [35]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [42]:
hyper_params = {
    'C': range(1, 21), # prev tests showed a high C was preferred
    'tol': [0.001, 0.01, 0.1],
    'class_weight': [None, 'balanced'], # balanced option helps with class frequencies, might be worth looking into
    'kernel': ['linear', 'rbf'],
    'random_state' : [SEED]
}
svc = svm.SVC()
search = GridSearchCV(estimator = svc, param_grid=hyper_params, refit = True, cv = 5)
search_results = search.fit(X, Y)

In [45]:
search_results.best_params_

{'C': 7,
 'class_weight': None,
 'kernel': 'rbf',
 'random_state': 42,
 'tol': 0.001}

In [46]:
search_results.best_score_

0.6507931173544831